<a href="https://colab.research.google.com/github/akpax/DeepLearningPractice/blob/main/CollabFiltering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 66.2 MB/s eta 0:00:00
Mounted at /content/gdrive


In [2]:
#hide
from fastbook import *


# Probabilistic Matrix Factorizing
Use Probabilistic Matrix Factorizing to predict movie recommendations. (essentailly a dot product approach)

---



In [3]:
from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

In [10]:
# pull data
ratings_df = pd.read_csv(path/"u.data", delimiter = "\t", header=None,
                      names=['user','movie','rating','timestamp'])
ratings_df.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [7]:
movies_df = pd.read_csv(path/'u.item',  delimiter='|', encoding='latin-1',
                     usecols=(0,1), names=('movie','title'), header=None)
movies_df.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [11]:
ratings_df = ratings_df.merge(movies_df, on="movie")
ratings_df.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [13]:
dls = CollabDataLoaders.from_df(ratings_df,item_name="movie")
dls.show_batch()

,user,movie,rating
0,13,89,4
1,37,550,4
2,135,79,3
3,774,418,2
4,498,100,3
5,807,144,4
6,577,1147,4
7,13,668,1
8,249,9,5
9,821,427,5


In [17]:
num_users = len(dls.classes["user"])
num_movies = len(dls.classes["movie"])

print(f"Number of users: {num_users}")
print(f"Number of movies: {num_movies}")

Number of users: 944
Number of movies: 1646


In [40]:
class DotProductModel(Module):
  def __init__(self, num_users, num_movies, num_factors, y_range=(0,5.5)):
    super().__init__()
    self.users = Embedding(num_users, num_factors)
    self.movies = Embedding(num_movies, num_factors)
    self.y_range = y_range

  def forward(self, x):
    users = self.users(x[:,0])
    movies = self.movies(x[:,1])
    return sigmoid_range((users*movies).sum(dim=1), *self.y_range)



In [41]:
model = DotProductModel(num_users, num_movies, 50)
learner = Learner(dls, model, loss_func=MSELossFlat())


In [42]:
learner.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.999460,0.981030,00:08
1,0.840408,0.892480,00:09
2,0.663646,0.859706,00:09
3,0.477997,0.865488,00:08
4,0.357738,0.869794,00:09


In [46]:
# add bias to DotProductModel
class DotProductModel(Module):
  def __init__(self, num_users, num_movies, num_factors, y_range=(0,5.5)):
    super().__init__()
    self.users = Embedding(num_users, num_factors)
    self.movies = Embedding(num_movies, num_factors)
    self.user_bias = Embedding(num_users, 1)
    self.movie_bias = Embedding(num_movies, 1)
    self.y_range = y_range

  def forward(self, x):
    users = self.users(x[:,0])
    movies = self.movies(x[:,1])
    res = (users*movies).sum(dim=1, keepdim=True)
    res += self.user_bias(x[:,0]) + self.movie_bias(x[:,0])
    return sigmoid_range(res, *self.y_range)



In [47]:
model = DotProductModel(num_users, num_movies, 50)
learner = Learner(dls, model, loss_func=MSELossFlat())


In [48]:
learner.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.937908,0.918576,00:12
1,0.834221,0.860182,00:10
2,0.673516,0.848798,00:09
3,0.433707,0.864524,00:10
4,0.303810,0.869697,00:10


In [50]:
## fit again but this time use weight decay (L2 Regulatization (weights squared))
model = DotProductModel(num_users, num_movies, 50)
learner = Learner(dls, model, loss_func=MSELossFlat())
learner.fit_one_cycle(5, 5e-3, wd=0.1)

# Model performs much better with L2 regularization

epoch,train_loss,valid_loss,time
0,0.950359,0.921436,00:09
1,0.871907,0.865960,00:08
2,0.779003,0.823159,00:09
3,0.613630,0.797542,00:09
4,0.532532,0.796456,00:09


# Deep Learning for Collaborative Filtering

In [52]:
embs = get_emb_sz(dls)
embs

[(944, 74), (1646, 101)]

In [54]:
embs[0][1]+embs[1][1]

175

In [65]:
class CollabNN(Module):
  def __init__(self, size_user, size_item, n_act, y_range=(0,5.5)):
    super().__init__()
    self.user_factors = Embedding(*size_user)
    self.movie_factors = Embedding(*size_item)
    self.layers = nn.Sequential(
        nn.Linear(size_user[1]+size_item[1],n_act),
        nn.ReLU(),
        nn.Linear(n_act,1)
    )
    self.y_range=y_range

  def forward(self,x):
    embs = self.user_factors(x[:,0]), self.movie_factors(x[:,1])
    x = self.layers(torch.cat(embs,dim=1))
    # print(f"User Factors: {embs[0].size()}")
    # print(f"Movie Factors: {embs[1].size()}")
    # print(f"torch.cat: {torch.cat(embs,dim=1).size()}")
    # print(x.size())
    return sigmoid_range(x, *self.y_range)

In [66]:
model = CollabNN(*embs, 100)
learner = Learner(dls, model, loss_func=MSELossFlat())
learner.fit_one_cycle(5, 5e-3)

epoch,train_loss,valid_loss,time
0,0.964832,0.937726,00:09
1,0.905173,0.884061,00:10
2,0.883539,0.862596,00:10
3,0.803323,0.852258,00:10
4,0.755592,0.853655,00:09
